In [9]:
# !pip install -U --user /kaggle/input/lama-nn/LightAutoML-0.3.7.4-py3-none-any.whl >> none

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
import copy
import gc
import os
import time
import lightautoml

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
# target = pd.read_csv("tr.csv", usecols=['user_id', 'is_male', 'age'])
# # test = pd.read_csv("/kaggle/input/mlcupmlfeaturesv6/test_ml_num_v2.csv", nrows=10000)
# # test.sort_values("user_id", inplace=True)

# print(len(target))

In [4]:
# oof_age_1 = pd.read_csv("oof_warm_age_lin_lgbm_automl.csv")
# oof_age_2 = pd.read_csv("oof_warm_age_big_automl.csv")

# test_age_1 = pd.read_csv("test_warm_age_lin_lgbm_automl.csv")
# test_age_2 = pd.read_csv("test_warm_age_big_automl.csv")

# pca_train = pd.read_csv("pca_train.csv")
# pca_test = pd.read_csv("pca_test.csv")

# fl_train = pd.read_csv("fl_train_11.csv")
# fl_test = pd.read_csv("fl_test_11.csv")

# train_age_dart_pred = pd.read_csv("train_age_dart_pred.csv")
# test_age_dart_pred = pd.read_csv("test_age_dart_pred.csv")

# age_gru_oof = pd.read_csv("age_gru_oof.csv")
# age_gru_test = pd.read_csv("age_gru_test.csv")

In [5]:
# oof_age = oof_age_1.merge(oof_age_2, on='user_id')
# test_age = test_age_1.merge(test_age_2, on='user_id')

# oof_age = oof_age.merge(pca_train, on='user_id')
# test_age = test_age.merge(pca_test, on='user_id')

# oof_age = oof_age.merge(fl_train, on='user_id')
# test_age = test_age.merge(fl_test, on='user_id')

# oof_age = oof_age.merge(train_age_dart_pred, on='user_id', how ='left')
# test_age = test_age.merge(test_age_dart_pred, on='user_id', how ='left')

# oof_age = oof_age.merge(age_gru_oof, on='user_id', how ='left')
# test_age = test_age.merge(age_gru_test, on='user_id', how ='left')

# oof_age.head()

In [6]:
# oof_age = oof_age.merge(target, on='user_id')
# print(len(oof_age))

In [8]:
# oof_age.to_csv("b/oof_age_st_lvl2.csv", index=False)
# test_age.to_csv("b/test_age_st_lvl2.csv", index=False)

oof_age = pd.read_csv("oof_age_st_lvl2_v3.csv").drop(columns=['Unnamed: 0'])
test_age = pd.read_csv("test_age_st_lvl2_v3.csv").drop(columns=['Unnamed: 0'])
oof_age.shape

(263065, 116)

In [8]:
rnn_oof = pd.read_csv("warm_age/age_lstm_oof_1024.csv")
print(rnn_oof.shape)
rnn_test = pd.read_csv("warm_age/age_lstm_test_1024.csv")
rnn_oof.head()

(263065, 8)


,age_prob_1024_lstm_0,age_prob_1024_lstm_1,age_prob_1024_lstm_2,age_prob_1024_lstm_3,age_prob_1024_lstm_4,age_prob_1024_lstm_5,age_prob_1024_lstm_6,user_id
0,2.117548e-09,0.007946,0.121810,0.417168,0.316269,0.119947,0.016860,0
1,3.110883e-07,0.012638,0.143254,0.399116,0.304523,0.124890,0.015578,1
2,1.338287e-07,0.303386,0.462054,0.172003,0.041937,0.014941,0.005680,2
3,7.894442e-08,0.058100,0.346862,0.409988,0.133644,0.044026,0.007381,3
4,5.248373e-08,0.278061,0.586396,0.114949,0.013987,0.005035,0.001573,4


In [9]:
oof_age = oof_age.merge(rnn_oof, on='user_id')
test_age = test_age.merge(rnn_test, on='user_id')
print(oof_age.shape)

(263065, 144)


In [10]:
# train = train[~train['is_male'].isna() & (train['is_male'].apply(lambda x: x != 'NA'))]
# train['is_male'] = train['is_male'].astype('int')
# print(len(train))

In [10]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task
import time


roles = {'target': 'age',
                 'drop': ['user_id', 'index', "is_male"]}
default_nn_params = {
    "n_epochs": 200,
}
default_nn_params_2 = {
    "n_epochs": 100,
}
automl = TabularAutoML(
    task = Task(name='multiclass', metric="f1_weighted"),
    cpu_limit = 24,
    timeout=3600*1500,
    gpu_ids='0',
    debug=True,
    general_params = {"use_algos": [["linear_l2", "dense", "resnet", "lgb"]]},
    nn_pipeline_params = {"use_te": True,  "max_intersection_depth": 0},
    reader_params = {'cv': 10, 'random_state': 43},
)
start_time = time.time()
oof_pred = automl.fit_predict(
        oof_age,
        roles=roles,
        verbose = 1
    )
end_time = time.time()

[20:00:43] Stdout logging level is INFO.
[20:00:43] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[20:00:43] Task: multiclass

[20:00:43] Start automl preset with listed constraints:
[20:00:43] - time: 5400000.00 seconds
[20:00:43] - CPU: 24 cores
[20:00:43] - memory: 16 GB

[20:00:43] Train data shape: (263065, 116)

[20:00:51] Layer 1 train process start. Time left 5399991.78 secs
[20:00:52] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[20:03:54] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.4931149062556765
[20:03:54] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[20:03:54] Time left 5399809.08 secs

[20:04:37] Selector_LightGBM fitting and predicting completed
[20:04:37] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[20:09:53] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.49322808027770076
[20:09:53] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
[20:09:53] Time left 5399449.89 se

In [ ]:
# 0.4942498264611808

In [11]:
print(f"spend time: {end_time - start_time}")
print(automl.create_model_str_desc())

spend time: 913.2793254852295
Final prediction for new objects (level 0) = 
	 0.29181 * (10 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.41455 * (10 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.15283 * (10 averaged models Lvl_0_Pipe_2_Mod_0_TorchNN_dense_0) +
	 0.14081 * (10 averaged models Lvl_0_Pipe_2_Mod_1_TorchNN_resnet_1) 


In [12]:
test_age.sort_values("user_id", inplace=True)
# class_mapping = {v:k for k, v in automl.outer_pipes[0].ml_algos[0].models[0][0].reader.class_mapping.items()}
test_pred_age_probs = automl.predict(test_age).data
# test_pred_age_probs = np.hstack([test_pred_age_probs[:, class_mapping[i]-1].reshape(-1, 1) for i in range(6)])
with open('stack_lvl2_age_warm_final_2.npy', 'wb') as f:
    np.save(f, test_pred_age_probs)

In [13]:
list(np.argmax(test_pred_age_probs, axis=1))

[0,
 5,
 3,
 0,
 5,
 4,
 5,
 5,
 0,
 5,
 0,
 5,
 5,
 3,
 5,
 0,
 3,
 5,
 0,
 5,
 5,
 5,
 0,
 5,
 0,
 5,
 0,
 0,
 5,
 0,
 3,
 5,
 0,
 0,
 3,
 0,
 2,
 0,
 0,
 3,
 5,
 5,
 5,
 5,
 3,
 5,
 0,
 0,
 5,
 0,
 5,
 5,
 0,
 0,
 5,
 0,
 5,
 5,
 5,
 5,
 5,
 5,
 1,
 5,
 0,
 0,
 5,
 5,
 0,
 0,
 5,
 5,
 5,
 0,
 3,
 0,
 3,
 5,
 0,
 0,
 5,
 5,
 5,
 5,
 0,
 3,
 5,
 0,
 5,
 3,
 0,
 0,
 5,
 5,
 0,
 5,
 5,
 0,
 0,
 5,
 5,
 3,
 5,
 5,
 0,
 5,
 5,
 5,
 5,
 0,
 5,
 5,
 3,
 5,
 3,
 5,
 3,
 5,
 0,
 0,
 3,
 0,
 0,
 5,
 5,
 5,
 0,
 5,
 3,
 0,
 0,
 0,
 3,
 5,
 0,
 5,
 5,
 5,
 0,
 3,
 3,
 5,
 0,
 0,
 0,
 5,
 0,
 5,
 0,
 0,
 3,
 5,
 0,
 5,
 0,
 3,
 0,
 5,
 0,
 5,
 3,
 5,
 3,
 5,
 5,
 5,
 5,
 1,
 0,
 0,
 3,
 5,
 3,
 3,
 0,
 5,
 5,
 0,
 5,
 5,
 0,
 5,
 5,
 5,
 0,
 5,
 5,
 0,
 0,
 5,
 3,
 0,
 0,
 0,
 0,
 5,
 5,
 0,
 0,
 5,
 5,
 0,
 0,
 5,
 0,
 3,
 0,
 0,
 5,
 0,
 0,
 5,
 3,
 5,
 3,
 3,
 5,
 5,
 5,
 5,
 0,
 0,
 0,
 3,
 5,
 0,
 0,
 0,
 5,
 5,
 0,
 5,
 0,
 5,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 5,
 5,
 0,


In [14]:
import collections

col = collections.Counter(list(np.argmax(test_pred_age_probs, axis=1)))

In [15]:
col

Counter({0: 42568, 5: 86325, 3: 15078, 4: 393, 2: 18, 1: 342})